<a href="https://colab.research.google.com/github/devLarissaRodrigues/WoMakersCode-DataAnalytics-DesafiosEmSquad/blob/main/Desafio6_completo_em_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio 6: Desafio de Classificação - Diagnóstico de Alzheimer com Dados Cognitivos

### Bootcamp Data Analytics – WoMakersCode  
👩🏽‍💻 Squad: Mae C. Jemison
### 👥 Integrantes:

- Amparo Silvia Pastor Castillo Runschka  
- Cristiana Alves Moura  
- Fernanda Monello Correa Manoel  
- Larissa Monique de Sousa Rodrigues  
- Raquel Nunes Matos Rios  
- Vanessa Lima Paraizo  
- Verônica Lima Costa  

## 📝 Descrição do Desafio

Contexto
Este desafio utiliza o conjunto de dados OASIS (Open Access Series of
Imaging Studies), que contém informações clínicas e demográficas de
pacientes com potencial risco de Alzheimer. O objetivo é prever se o
paciente apresenta sinais de demência.
A variável alvo é binária:
* 0: Sem demência
* 1: Com demência

📁 *Este notebook contém todas as etapas da análise de dados desenvolvida para o desafio.*

## 📚 Importação de Bibliotecas

Antes de começar, vamos importar as bibliotecas necessárias para manipulação, análise e visualização dos dados.

In [1]:
# Manipulação e análise de dados
import pandas as pd
import numpy as np

# Visualização
import seaborn as sns
import matplotlib.pyplot as plt

# Configurações estéticas para os gráficos
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Estatística
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

### 📁 Visualização Inicial

Nesta etapa, vamos observar as primeiras linhas do dataset para entender sua estrutura, as colunas disponíveis e o formato dos dados.

Nosso objetivo aqui é ter uma visão geral antes de qualquer limpeza ou tratamento.

In [2]:
# Configurando URL a partir do id do arquivo compartilhado

id_do_arquivo = "1FmLKaVzvG7YD8dwMFlw-W5ZuGCbuRcPb"
url = f'https://drive.google.com/uc?id={id_do_arquivo}&export=download'
df = pd.read_csv(url)
df.head()

,Age,Sex,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF,Group
0,66,M,19,5,21,1.0,1474,0.768,1.11,0
1,79,M,14,5,22,2.0,1630,0.722,0.82,0
2,88,M,8,5,22,0.0,1432,0.668,1.24,1
3,74,M,14,3,24,1.0,1276,0.833,1.29,0
4,70,M,18,4,25,0.0,1605,0.677,1.28,0


### 📉 Verificação de Valores Ausentes e Duplicatas

Nesta etapa, vamos verificar se existem valores ausentes ou registros duplicados no dataset.

Esses problemas podem impactar os resultados das análises estatísticas e visualizações, e devem ser tratados ainda na fase de pré-processamento.

In [4]:
# Verificar tipos de dados e valores nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Age     200 non-null    int64  
 1   Sex     200 non-null    object 
 2   EDUC    200 non-null    int64  
 3   SES     200 non-null    int64  
 4   MMSE    200 non-null    int64  
 5   CDR     200 non-null    float64
 6   eTIV    200 non-null    int64  
 7   nWBV    200 non-null    float64
 8   ASF     200 non-null    float64
 9   Group   200 non-null    int64  
dtypes: float64(3), int64(6), object(1)
memory usage: 15.8+ KB


In [6]:
# Verificar valores ausentes
print("🔍 Valores ausentes por coluna:")
print(df.isnull().sum())

# Verificar registros duplicados
duplicatas = df.duplicated().sum()
print(f"\n📦 Registros duplicados encontrados: {duplicatas}")

🔍 Valores ausentes por coluna:
Age      0
Sex      0
EDUC     0
SES      0
MMSE     0
CDR      0
eTIV     0
nWBV     0
ASF      0
Group    0
dtype: int64

📦 Registros duplicados encontrados: 0


In [7]:
# Obtenção de dados estatísticos
df.describe()

,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF,Group
count,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,74.820000,12.335000,3.110000,19.57500,0.830000,1494.100000,0.757850,1.048050,0.270000
std,9.094015,4.214449,1.469249,5.93807,0.730595,104.309358,0.060322,0.148604,0.445074
min,60.000000,6.000000,1.000000,10.00000,0.000000,1234.000000,0.651000,0.800000,0.000000
25%,67.000000,8.750000,2.000000,14.00000,0.000000,1420.000000,0.702000,0.910000,0.000000
50%,74.000000,12.000000,3.000000,20.00000,0.500000,1497.500000,0.767000,1.040000,0.000000
75%,83.000000,16.000000,4.000000,25.00000,1.000000,1559.250000,0.813250,1.170000,1.000000
max,89.000000,19.000000,5.000000,29.00000,2.000000,1764.000000,0.850000,1.300000,1.000000


In [8]:
# Selecionando variáveis numéricas de acordo com os tipos:

# Seleciona apenas colunas numéricas
numerico_df = df.select_dtypes(include=['float64', 'int64'])

# Exibe os nomes das colunas numéricas
numerico_df.columns

Index(['Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF', 'Group'], dtype='object')

In [9]:
# Calculando a correlação de Pearson

# Selecionando variáveis numéricas de acordo com os tipos:
numerico_df = df.select_dtypes(include=['float64', 'int64'])

# Calcula a correlação de todas as variáveis numéricas
correlations = numerico_df.corr()
# Exibe as correlações
print(correlations)

            Age      EDUC       SES      MMSE       CDR      eTIV      nWBV  \
Age    1.000000  0.022560  0.023303 -0.095969  0.016927 -0.037005 -0.066078   
EDUC   0.022560  1.000000  0.025669  0.045074  0.085502  0.000758 -0.005909   
SES    0.023303  0.025669  1.000000  0.084870  0.090070  0.052259  0.110977   
MMSE  -0.095969  0.045074  0.084870  1.000000 -0.134885 -0.040504 -0.020970   
CDR    0.016927  0.085502  0.090070 -0.134885  1.000000 -0.049527  0.064811   
eTIV  -0.037005  0.000758  0.052259 -0.040504 -0.049527  1.000000  0.027471   
nWBV  -0.066078 -0.005909  0.110977 -0.020970  0.064811  0.027471  1.000000   
ASF    0.031457  0.089791  0.081542  0.177585 -0.034311 -0.104985  0.002546   
Group -0.108361  0.050660  0.154153  0.036031 -0.020399  0.063927 -0.073165   

            ASF     Group  
Age    0.031457 -0.108361  
EDUC   0.089791  0.050660  
SES    0.081542  0.154153  
MMSE   0.177585  0.036031  
CDR   -0.034311 -0.020399  
eTIV  -0.104985  0.063927  
nWBV   0.0025